## 4 AIS dataset process

In [163]:
to_process_data_path = r'W:\Neeraj_thesis\version2_thesis\dataset\2019\generator_version_3_output\data_2019_01_v3_out.csv' # NOT USE Version_4: as is for transformer set 
to_store_data_path_X = r'W:\Neeraj_thesis\version2_thesis\dataset\2019\generator_version_5_output\data_2019_12_v5_out\data_2019_12_X.npy'
to_store_data_path_Y_lat = r'W:\Neeraj_thesis\version2_thesis\dataset\2019\generator_version_5_output\data_2019_12_v5_out\data_2019_12_Y_lat.npy'
to_store_data_path_Y_lon = r'W:\Neeraj_thesis\version2_thesis\dataset\2019\generator_version_5_output\data_2019_12_v5_out\data_2019_12_Y_lon.npy'
to_store_data_path_Y = r'W:\Neeraj_thesis\version2_thesis\dataset\2019\generator_version_5_output\data_2019_01_v5_out\data_2019_01_Y.npy'


In [164]:
# import libraries

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import figure
from matplotlib.colors import ListedColormap
from matplotlib import cm

from math import radians, sin, cos, sqrt, atan2

In [165]:
ais_std_v1 = pd.read_csv(to_process_data_path)

In [168]:
# closest_dist_shore
min_dist_coast = ais_std_v1['NDC_nm'].min()
max_dist_coast = ais_std_v1['NDC_nm'].max()
print('min_to_coast (in nm)', min_dist_coast)
print('max_to_coast (in nm)', max_dist_coast)

if max_dist_coast > (50000/1852):
    raise ValueError('Maximum distance to coast is greater than 26.99 nm')
else:
    pass

ais_std_v2 = ais_std_v1.copy()

min_to_coast (in nm) 0.0
max_to_coast (in nm) 26.811


In [169]:
ais_std_v2 = ais_std_v2.drop(['mmsi','cog','unixtime','short_voyage_id'], axis=1).reset_index(drop = True)

In [170]:
ais_std_v3 = ais_std_v2.copy()

# define the min and max values for lat and lon standardization
min_lat, max_lat = 58.6, 59.93
min_lon, max_lon = 9.4, 11.45
min_sog, max_sog = 0 , 40
min_coast, max_coast = 0, round(50000/1852, 3)

# standardize the lat and lon values between 0 and 1
ais_std_v3['lat_s'] = (ais_std_v3['lat'] - min_lat) / (max_lat - min_lat)
ais_std_v3['lon_s'] = (ais_std_v3['lon'] - min_lon) / (max_lon - min_lon)
ais_std_v3['sog_s'] = (ais_std_v3['sog'] - min_sog) / (max_sog - min_sog)
ais_std_v3['NDC_s'] = (ais_std_v3['NDC_nm'] - min_coast) / (max_coast - min_coast)

ais_std_v3 = ais_std_v3.drop(['lat','lon','sog','NDC_nm'], axis=1).reset_index(drop = True)

# new column arrangement
col_arrange = ['lat_s','lon_s','sog_s','cog_sin','cog_cos','NDC_s','In_TSS', 'Vessel_TSS', 'new_short_voyage_id','position']
ais_std_v3 = ais_std_v3[col_arrange]

In [171]:
# replace NaN values in Vessel_TSS column with zeros
ais_std_v3['Vessel_TSS'].fillna(0, inplace=True)
ais_std_v3.isna().sum()

lat_s                  0
lon_s                  0
sog_s                  0
cog_sin                0
cog_cos                0
NDC_s                  0
In_TSS                 0
Vessel_TSS             0
new_short_voyage_id    0
position               0
dtype: int64

In [172]:
# Prepare dataset for LSTM,CNN-lstm,GRU Models

def create_datasets(data, voyage_col_name , look_back = 10, target_size = 10):
    voyage_tags = data[voyage_col_name].unique()

    X, Y_lat, Y_lon, Y = [],[],[],[]
    for voyage in voyage_tags:
        voyage_df = data[data [voyage_col_name] == int(voyage)] 
        voyage_df.sort_values('position')
        voyage_df_v2 = voyage_df[['lat_s', 'lon_s', 'sog_s', 'cog_sin' , 'cog_cos', 'NDC_s', 'In_TSS', 'Vessel_TSS']].values

        for i in range(len(voyage_df_v2) - look_back -target_size + 1): # eg. for len(data) = 39 ; range (20)
            X.append(voyage_df_v2[i:(i+look_back)]) # Starts from [0:10] ends at [19:29] for a len(data) = 30
            Y_lat.append(voyage_df_v2[(i+look_back) : (i+look_back+target_size), 0:1]) # Starts at [10:20] ends at [29:39]
            Y_lon.append(voyage_df_v2[(i+look_back) : (i+look_back+target_size), 1:2]) # Starts at [10:20] ends at [29:39]
            Y.append(voyage_df_v2[(i+look_back) : (i+look_back+target_size), 0:2]) # Starts at [10:20] ends at [29:39]

    return np.array(Y), np.array(X), np.array(Y_lat), np.array(Y_lon)

In [173]:
# data_X, data_Y_lat, data_Y_lon = create_datasets( ais_std_v3, voyage_col_name= 'new_short_voyage_id', look_back= 10, target_size= 10 ) # 10 steps -> 5 minutes

data_Y = create_datasets( ais_std_v3, voyage_col_name= 'new_short_voyage_id', look_back= 10, target_size= 10 ) # 10 steps -> 5 minutes

In [174]:
# Save the datasets as NumPy arrays : with 6 key features standardized ('lat', 'lon', 'sog', 'cog_sin', 'cog_cos', 'NDC_nm', in_TSS, Vessel_TSS  )
np.save(to_store_data_path_X, data_X)
np.save(to_store_data_path_Y_lat, data_Y_lat)
np.save(to_store_data_path_Y_lon, data_Y_lon)
np.save(to_store_data_path_Y, data_Y)